In [1]:
import sparcl.client

In [2]:
client = sparcl.client.SparclClient(url=sparcl.client._DEV, verbose=True)
client 

/tmp/ipykernel_88056/1540030723.py:1: UserWarning: Implementation ignores that fact that a
        single Science Field Name might map to MULTIPLE Internal Field Names
        within a single Data Set.  If this is the case (see Admin)
        results may be unpredictable!!!
  client = sparcl.client.SparclClient(url=sparcl.client._DEV, verbose=True)


(sparclclient:0.4.0a5.dev4, api:6.0, http://localhost:8030/sparc, verbose=True, connect_timeout=1.1, read_timeout=5400.0)

In [3]:
client.get_default_fields(None)

{'flux', 'uuid', 'wavelength'}

In [4]:
client.get_default_fields('BOSS-DR16')

['uuid', 'flux', 'wavelength']

In [5]:
print(sorted(client.dr_o2n['DESI-edr'].keys()))  # No PLATEID, but is in dr_all['DESI-edr']

['FIBER', 'HEALPIX', 'LOCATION', 'MJD', 'PROGRAM', 'RUN2D', 'SURVEY', 'SV_PRIMARY', 'TARGETID', 'TILEID', 'data_release', 'datasetgroup', 'dateobs', 'dateobs_center', 'dec', 'dirpath', 'exptime', 'extra_files', 'filename', 'filesize', 'flux', 'id', 'instrument', 'ivar', 'mask', 'model', 'ra', 'redshift', 'redshift_err', 'redshift_warning', 'site', 'sky', 'specid', 'specprimary', 'spectype', 'targetid', 'telescope', 'wave_sigma', 'wavelength', 'wavemax', 'wavemin']


In [6]:
print(sorted(client.dr_all.get('DESI-edr')))

['FIBER', 'HEALPIX', 'LOCATION', 'MJD', 'PROGRAM', 'RUN2D', 'SURVEY', 'SV_PRIMARY', 'TARGETID', 'TILEID', 'data_release', 'datasetgroup', 'dateobs', 'dateobs_center', 'dec', 'exptime', 'flux', 'id', 'instrument', 'ivar', 'mask', 'model', 'ra', 'redshift', 'redshift_err', 'redshift_warning', 'site', 'sky', 'specid', 'specprimary', 'spectype', 'targetid', 'telescope', 'wave_sigma', 'wavelength', 'wavemax', 'wavemin']


In [7]:
print(sorted(client.get_all_fields('DESI-edr')))

['FIBER', 'HEALPIX', 'LOCATION', 'MJD', 'PROGRAM', 'RUN2D', 'SURVEY', 'SV_PRIMARY', 'TARGETID', 'TILEID', 'data_release', 'datasetgroup', 'dateobs', 'dateobs_center', 'dec', 'exptime', 'flux', 'instrument', 'ivar', 'mask', 'model', 'ra', 'redshift', 'redshift_err', 'redshift_warning', 'site', 'sky', 'specid', 'specprimary', 'spectype', 'targetid', 'telescope', 'uuid', 'wave_sigma', 'wavelength', 'wavemax', 'wavemin']


In [8]:
print(sorted(client.get_all_fields('BOSS-DR16'))) # NOTE: redshift => z

['FIBERID', 'MJD', 'PLATEID', 'RUN1D', 'RUN2D', 'SPECOBJID', 'data_release', 'datasetgroup', 'dateobs', 'dateobs_center', 'dec', 'exptime', 'flux', 'instrument', 'ivar', 'mask', 'model', 'ra', 'redshift', 'redshift_err', 'redshift_warning', 'site', 'sky', 'specid', 'specprimary', 'spectype', 'targetid', 'telescope', 'uuid', 'wave_sigma', 'wavelength', 'wavemax', 'wavemin']


In [9]:
print(client.dr_o2n['DESI-edr'])

{'wavelength': 'wavelength', 'SV_PRIMARY': 'SV_PRIMARY', 'wavemax': 'wavemax', 'telescope': 'telescope', 'flux': 'flux', 'wave_sigma': 'wave_sigma', 'HEALPIX': 'HEALPIX', 'sky': 'sky', 'dateobs': 'dateobs', 'ivar': 'ivar', 'data_release': 'data_release', 'datasetgroup': 'datasetgroup', 'FIBER': 'FIBER', 'TARGETID': 'TARGETID', 'SURVEY': 'SURVEY', 'specprimary': 'specprimary', 'instrument': 'instrument', 'filename': 'filename', 'extra_files': 'extra_files', 'RUN2D': 'RUN2D', 'targetid': 'targetid', 'dirpath': 'dirpath', 'dec': 'dec', 'spectype': 'spectype', 'MJD': 'MJD', 'model': 'model', 'exptime': 'exptime', 'TILEID': 'TILEID', 'filesize': 'filesize', 'dateobs_center': 'dateobs_center', 'ra': 'ra', 'redshift_warning': 'redshift_warning', 'specid': 'specid', 'site': 'site', 'LOCATION': 'LOCATION', 'id': 'uuid', 'mask': 'mask', 'wavemin': 'wavemin', 'redshift_err': 'redshift_err', 'redshift': 'redshift', 'PROGRAM': 'PROGRAM'}


In [10]:
# get sample ids
found = client.find(['uuid', 'data_release'], limit=50)
ids = found.ids
found.info

{'META': {'endpoint': 'sparc/find'},
 'PARAMETERS': {'limit': 50,
  'include': None,
  'offset': 0,
  'json_payload': {'outfields': ['id', 'data_release'], 'search': []}},
 'HEADER': {'data_release': 'category', 'id': 'str'},
 'WARNINGS': ["OFFSET parameter needs SORT but it was not provided. Using default 'id' for sorting"],
 'Count': 6}

In [12]:
# orig=redshift, new=z in DataFields
dr='DESI-edr'
id_desi = [rec['uuid'] for rec in found.records if rec.get('data_release') == dr]
print(f'Found {len(id_desi)} ids from Data Set = {dr}')
results = client.retrieve(id_desi, include=['flux', 'wavelength', 'uuid'], dataset_list=[dr])
results.info

Found 2 ids from Data Set = DESI-edr
Using url="http://localhost:8030/sparc/retrieve/?include=id%2Cflux%2Cwavelength&dataset_list=DESI-edr"
Got response to post in 0.09353263399680145 seconds
Got 2 spectra in 0.09 seconds (21 spectra/sec)
{'success': True, 'info': 'Successfully found 2 records in dr=DESI-edr'}


{'status': {'success': True,
  'info': 'Successfully found 2 records in dr=DESI-edr'},
 'Count': 2}

In [13]:
print(f'dict key access: {len(results.records[0]["flux"])}')
print(f'dict DOT access: {len(results.records[0].flux)}')

dict key access: 7781
dict DOT access: 7781


In [14]:
import sparcl.utils as ut

In [15]:
dd = ut._AttrDict(results.records[0])

In [16]:
dd._dr

'DESI-edr'

In [21]:
specids = [1429831265344501, 1429831265410037]
results = client.retrieve_by_specid(specids, include=['uuid','specid'], verbose=True)
print(results.info)
results.records

Found 2 matches.
Using url="http://localhost:8030/sparc/retrieve/?include=id%2Cspecid&dataset_list=DESI-edr%2CBOSS-DR16%2CSDSS-DR16"
Got response to post in 0.01697138699819334 seconds
Got 2 spectra in 0.02 seconds (118 spectra/sec)
{'success': True, 'info': 'Successfully found 2 records in dr=DESI-edr,BOSS-DR16,SDSS-DR16'}
Got 2 records.
{'status': {'success': True, 'info': 'Successfully found 2 records in dr=DESI-edr,BOSS-DR16,SDSS-DR16'}, 'Count': 2}


[{'specid': 1429831265344501,
  'uuid': '4d62c9bc-f72b-47b3-a11e-0231a6e7e4a2',
  '_dr': 'BOSS-DR16'},
 {'specid': 1429831265410037,
  'uuid': 'fbef651a-e6cb-48b5-a1a9-2a3860f2c732',
  '_dr': 'BOSS-DR16'}]

# TODO
## Join request and response on id
## make sure AttrDict still works (dot notation for records to get fields)

## Use Case: Favorite Data Set list (ID Priority)
I have my **favorite DR list**. I use `client.find()` over each DR separately because each DR has a different special field I use in my constraints. I combine multi-find results into one `id_list`.  I only need fields that are common to all my DRs.  Using
   `results = client.retrieve(id_list)`
   with *ID Priority*, I get `results` that contains a list of homogeneous records.


In [ ]:
results.records[0].keys()

## Use Case: Favorite Fields (Field Priority)
 I have my **favorite fields**.  I need all records I get to have all
   of my favorite fields. I use one or more
   FIND commands to produce an id_list. Using
   `results = client.retrieve(id_list, include=fav_fields)`
   with *Field Priority*, I get `results` that contains a list of homogeneous
   records, all of which contain the full list of my fav_fields.
   I am pleased that I didn't have to think about any specific
   DR(s). I know that if my fav_fields had included too much, my
   RETRIEVE would have failed to find any records that included all
   fields.  As it is, checking `results.records` shows that only 70%
   of the ids I started with resulted in a record. I see which ids
   from my id_list were not used by looking at `results.status`.

## Use Case: Required plus Optional fields
I don't care about DRs but there are **some fields that are required** for
   my work. **Other fields are optional**.  I want them when they are
   available. I get the list of DRs that contain my required fields
   `dr_list = client.drs_common(req_fields)`
   Then I find records  from these DRs via:
`client.find(dr_list, ...)`
   For my work it  would be much easier to deal with homogeneous records, so I compute
   the full list of fields for my records with
`field_list = req_fields.intersect(opt_fields, client.fields_common(dr_list))`
   Then I get the records:
`results = client.retrieve(ids, field_list)`

## Use Case: Complex find yielding Hetergeneous records
I have ids from a complex combination of `client.find()` and DataLab queries. I must have the records associated with all ids.

   I want to process a homogeneous set of records using the largest
   set of common fields I can.  I'm anticipating doing fancy analysis
   with special fields but I don't exactly what that will entail yet,
   so I don't want to throw any fields out.

   There are *uncommon fields* I've found to be useful.  I want to
   process the records that have these uncommon fields seperately from
   the common batch.  I realize all records containing the uncommon
   fields will have ids that are also associatated with my common
   batch. I'll join the two batches using the id.
```
 found = client.find(...)  # multiple complex FIND and Data Lab queries
 ids = found.ids
 dr_list = found.dr_list
 common_fields = client.common_fields(dr_list)
 results_common = client.retrieve(ids, include=common_fields)
 result_uncommon = client.retrieve(ids, include=uncommon_fields)
```